In [192]:
import pandas as pd
import numpy as np
import re
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

import copy

from gensim.models import Word2Vec
from sklearn.cluster import KMeans


In [32]:
#word2vec libraries
import pandas as pd
import numpy as np
import re
import copy
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from pattern.text.en import singularize
import matplotlib.pyplot as plt

## Ingredientes recomendados que hacen falta para x receta
La idea es que el cliente también pueda buscar qué productos le hacen falta para realizar la receta que quiere

In [3]:
#Cargamos un dataset diferente:
ingredients_recipes_listas = pd.read_csv("ingredients_recipes_prueba_ingredientes_listas.csv")

In [4]:
ingredients_recipes_listas

,Unnamed: 0,title,ingredients_2,ingredients_3
0,0,"Lentil, Apple, and Turkey Wrap","['chicken', 'lentil', 'lentil', 'celery', 'pee...","['lowsodium vegetable chicken stock', 'dried b..."
1,1,Boudin Blanc Terrine with Red Onion Confit,"['cream', 'onion', 'salt', 'bay leaf', 'clove'...","['whipping cream', 'onion', 'salt', 'bay leaf'..."
2,2,Potato and Fennel Soup Hodge,"['fennel', 'onion', 'butter', 'potato', 'chick...",['fennel bulb sometimes called anise stalks di...
3,3,Mahi-Mahi in Tomato Olive Sauce,"['oil', 'onion', 'dry white wine', 'anchovy', ...","['extravirgin olive oil', 'onion', 'dry white ..."
4,4,Spinach Noodle Casserole,"['spinach', 'egg', 'cream', 'sauce', 'nutmeg',...","['ounce package frozen spinach souffl thawed',..."
...,...,...,...,...
20106,20125,Parmesan Puffs,"['egg', 'parmigianoreggiano finely grated', 'o...","['egg white', 'parmigianoreggiano finely grate..."
20107,20126,Artichoke and Parmesan Risotto,"['chicken', 'divided', 'oil', 'onion', 'baby a...","['lowsalt chicken broth', 'butter divided', 'e..."
20108,20127,Turkey Cream Puff Pie,"['tomato', 'onion', 'butter', 'salt', 'white m...","['tomato', 'onion finely', 'unsalted butter', ..."
20109,20128,Snapper on Angel Hair with Citrus Cream,"['oil', 'shallots thinly', 'clove', 'clam', 'c...","['olive oil', 'shallots thinly', 'garlic clove..."


In [5]:
type(ingredients_recipes_listas['ingredients_2'][0])

str

In [6]:
#al guardar el archivo en csv y cargarlo, las listas quedan almacenadas como strings
ingredients_recipes_listas['ingredients_2'][0]


"['chicken', 'lentil', 'lentil', 'celery', 'peeled', 'thyme', 'salt', 'tomato', 'apple', 'lemon', 'oil', 'pepper', 'tortilla', 'turkey', 'lettuce']"

In [7]:
#Eliminamos los caracteres especiales y separamos productos para que sea una lista
ingredients_recipes_listas['ingredients_2'] = [re.sub(r'[^a-zA-Z ]+', '', lista) for lista in ingredients_recipes_listas['ingredients_2']]
ingredients_recipes_listas['ingredients_2'] = [lista.split() for lista in ingredients_recipes_listas['ingredients_2']]

In [8]:
ingredients_recipes_listas['ingredients_2']

0        [chicken, lentil, lentil, celery, peeled, thym...
1        [cream, onion, salt, bay, leaf, clove, clove, ...
2           [fennel, onion, butter, potato, chicken, milk]
3        [oil, onion, dry, white, wine, anchovy, oregan...
4        [spinach, egg, cream, sauce, nutmeg, grated, s...
                               ...                        
20106       [egg, parmigianoreggiano, finely, grated, oil]
20107    [chicken, divided, oil, onion, baby, artichoke...
20108    [tomato, onion, butter, salt, white, mushrooms...
20109    [oil, shallots, thinly, clove, clam, cream, wh...
20110    [ham, clove, water, marmalade, cream, sugar, o...
Name: ingredients_2, Length: 20111, dtype: object

In [9]:
type(ingredients_recipes_listas['ingredients_2'][0])

list

### Ahora si revisamos qué ingredientes nos faltan para determinada receta
Lo ideal es que regrese los ingredientes tal como aparecen en el dataset de recetas (con detalle)

In [129]:
ingredients_recipes_listastest = ingredients_recipes_listas.loc[ingredients_recipes_listas['title']=='Lentil, Apple, and Turkey Wrap ']['ingredients_2']

In [130]:
ingredients_recipes_listastest = ingredients_recipes_listas[ingredients_recipes_listas['title']=='Lentil, Apple, and Turkey Wrap ']['ingredients_2']

### Tenemos que ver cómo convertimos lo anterior a lista úTIL

In [131]:
ingredients_recipes_listastest.tolist()

[['chicken',
  'lentil',
  'lentil',
  'celery',
  'peeled',
  'thyme',
  'salt',
  'tomato',
  'apple',
  'lemon',
  'oil',
  'pepper',
  'tortilla',
  'turkey',
  'lettuce']]

In [132]:
type(ingredients_recipes_listastest[0])

list

In [133]:
#Lista de prueba.... FUNCIONÓ
listax = ['chicken',
  'lentil',
  'lentil',
  'celery',
  'peeled',
  'thyme',
  'salt',
  'tomato',
  'apple',
  'lemon',
  'oil',
  'pepper',
  'tortilla',
  'turkey',
  'lettuce']

In [134]:
listax

['chicken',
 'lentil',
 'lentil',
 'celery',
 'peeled',
 'thyme',
 'salt',
 'tomato',
 'apple',
 'lemon',
 'oil',
 'pepper',
 'tortilla',
 'turkey',
 'lettuce']

In [135]:
ingredients_recipes_listastest = type(ingredients_recipes_listastest)

In [136]:
#Output es una lista de ingredientes que están faltan para la receta que el cliente quiera preparar
list(set(listax) - set(ingredientes_disponibles))

['peeled', 'lettuce', 'thyme', 'turkey', 'tortilla', 'lentil', 'celery']

In [137]:
#ingredients_recipes_listastest = ingredients_recipes_listas.loc[ingredients_recipes_listas['title']=='Lentil, Apple, and Turkey Wrap ']['ingredients_2']
#type(ingredients_recipes_listastest.tolist())
#ingredients_recipes_listastest.tolist()

In [138]:
#ingredients_recipes_listastest

El primer paso es que el cliente seleccione la receta

In [139]:
recomended_recipes

,disponibilidad
title,
Fully Salted Roast Chicken,1.0
Clarified Butter,1.0
Simple Lemon Dressing,1.0
Coconut Dulce de Leche,1.0
Tomato Butter,1.0
...,...
D.i.y. Drinks Bar,0.0
Daiquiri,0.0
Damn the Weather I,0.0


In [140]:
ingredients_recipes

,title,ingredients_2,disponibilidad
0,"Lentil, Apple, and Turkey Wrap",chicken,1
1,"Lentil, Apple, and Turkey Wrap",lentil,0
2,"Lentil, Apple, and Turkey Wrap",celery,0
3,"Lentil, Apple, and Turkey Wrap",peeled,0
4,"Lentil, Apple, and Turkey Wrap",thyme,0
...,...,...,...
164467,Baked Ham with Marmalade-Horseradish Glaze,water,0
164468,Baked Ham with Marmalade-Horseradish Glaze,marmalade,0
164469,Baked Ham with Marmalade-Horseradish Glaze,cream,0
164470,Baked Ham with Marmalade-Horseradish Glaze,sugar,1
